4 models each one with different env settings. \
Every model trained with Curriculum Learning: 6x6+8x8 or 6x6+8x8+10x10. \
Then a matrix sequence is shown.

# Env 1: Basic, DronePlacementEnv, Static grid. Result≈8steps

In [1]:
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from gymnasium import Env, spaces
import torch

UNEXPLORED = -1
OBSTACLE = 0
SAFE = 1

def drone_scan(drone_pos, scan_range, actual_env):
    half_range = scan_range // 2
    local_info = np.full((scan_range, scan_range), UNEXPLORED)
    grid_h, grid_w = actual_env.shape

    for i in range(scan_range):
        for j in range(scan_range):
            global_x = drone_pos[0] - half_range + i
            global_y = drone_pos[1] - half_range + j
            if 0 <= global_x < grid_h and 0 <= global_y < grid_w:
                local_info[i, j] = actual_env[global_x, global_y]

    return local_info, (drone_pos[0] - half_range, drone_pos[1] - half_range)

def stitch_information(global_grid, local_info, top_left):
    x_offset, y_offset = top_left
    grid_h, grid_w = global_grid.shape

    for i in range(local_info.shape[0]):
        for j in range(local_info.shape[1]):
            x, y = x_offset + i, y_offset + j
            if 0 <= x < grid_h and 0 <= y < grid_w:
                if global_grid[x, y] == UNEXPLORED:
                    global_grid[x, y] = local_info[i, j]
                elif global_grid[x, y] != local_info[i, j]:
                    if local_info[i, j] == SAFE:
                        global_grid[x, y] = SAFE
    return global_grid

class DronePlacementEnv(Env):
    def __init__(self, grid_size=10, max_steps=50):
        super().__init__()
        self.grid_size = grid_size
        self.max_steps = max_steps
        self.current_step = 0

        self.action_space = spaces.Discrete(grid_size * grid_size * 2)
        self.observation_space = spaces.Box(low=UNEXPLORED, high=SAFE, shape=(grid_size, grid_size), dtype=np.int32)

        self.reset()

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.actual_env = np.random.choice([OBSTACLE, SAFE], size=(self.grid_size, self.grid_size), p=[0.2, 0.8]).astype(np.int32)
        self.global_grid = np.full((self.grid_size, self.grid_size), UNEXPLORED, dtype=np.int32)
        self.current_step = 0
        return self.global_grid.copy(), {}

    def step(self, action):
        self.current_step += 1
        action = int(min(action, self.grid_size * self.grid_size * 2 - 1))
        action_per_row = self.grid_size * 2
        x = action // action_per_row
        y = (action % action_per_row) // 2
        drone_type = action % 2

        if not (0 <= x < self.grid_size and 0 <= y < self.grid_size):
            raise ValueError(f"Decoded position ({x}, {y}) is out of bounds!")

        scan_range = 3 if drone_type == 0 else 5
        local_info, top_left = drone_scan((x, y), scan_range, self.actual_env)
        prev_unexplored = np.sum(self.global_grid == UNEXPLORED)
        self.global_grid = stitch_information(self.global_grid, local_info, top_left)
        new_unexplored = np.sum(self.global_grid == UNEXPLORED)

        reward = float(prev_unexplored - new_unexplored - 0.2 - 0.2 * drone_type)
        if x <= 1 or x >= self.grid_size - 2 or y <= 1 or y >= self.grid_size - 2:
            reward += 0.3

        terminated = bool(new_unexplored == 0)
        truncated = bool(self.current_step >= self.max_steps)
        if terminated:
            reward += 10
        elif truncated:
            reward -= 5

        return self.global_grid.copy(), reward, terminated, truncated, {}

    def render(self):
        print(self.global_grid)

class SmallDroneEnv(DronePlacementEnv):
    def __init__(self):
        super().__init__(grid_size=6, max_steps=8)

In [2]:
def save_small_model_encoder():
    env = SmallDroneEnv()
    model = PPO("MlpPolicy", env, verbose=0)
    model.learn(total_timesteps=200000)

    # The progressive training strategy extracts the weights of the middle layer (hidden layer)
    encoder_state = {
        k: v for k, v in model.policy.mlp_extractor.state_dict().items()
        if "1" in k or "2" in k  # Only retain the parameters of the second layer and beyond (skip the input layer)
    }
    torch.save(encoder_state, "small_encoder_hidden.pt")
    model.save("ppo_small_model_1")

    return model

def train_large_model_with_transfer():
    env = DronePlacementEnv()

    policy_kwargs = dict(net_arch=[dict(pi=[64, 64], vf=[64, 64])])
    model = PPO("MlpPolicy", env, policy_kwargs=policy_kwargs, verbose=1)

    # Load hidden layer parameters
    encoder_weights = torch.load("small_encoder_hidden.pt")
    model.policy.mlp_extractor.load_state_dict(encoder_weights, strict=False)

    model.learn(total_timesteps=200000)
    model.save("ppo_large_model_1")


if __name__ == "__main__":
    save_small_model_encoder()
    train_large_model_with_transfer()    # Migrate the encoder to the large map to continue training

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 34.9     |
|    ep_rew_mean     | 105      |
| time/              |          |
|    fps             | 702      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 36.9        |
|    ep_rew_mean          | 103         |
| time/                   |             |
|    fps                  | 568         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.022465367 |
|    clip_fraction        | 0.205       |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.28       |
|    explained_variance   | 0.022       |
|    learning_rate        | 0.

In [3]:
def run_trained_model():
    model = PPO.load("ppo_large_model_1")
    env = DronePlacementEnv()
    obs, _ = env.reset()

    for step in range(env.max_steps):
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, _ = env.step(action)
        x = action // 20
        y = (action % 20) // 2
        drone_type = "3x3" if action % 2 == 0 else "5x5"
        print(f"Step {step}: Placed {drone_type} drone at ({x}, {y}), reward: {reward:.2f}")
        env.render()
        if terminated or truncated:
            print("Mission Complete: All cells explored!")
            break
        if truncated:
            print("Max steps reached.")
            break

if __name__ == "__main__":
    run_trained_model()

Step 0: Placed 5x5 drone at (3, 3), reward: 24.60
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  1  0  1  0 -1 -1 -1 -1]
 [-1  0  1  0  1  1 -1 -1 -1 -1]
 [-1  1  1  1  1  1 -1 -1 -1 -1]
 [-1  1  1  1  1  1 -1 -1 -1 -1]
 [-1  1  1  0  1  0 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]
Step 1: Placed 5x5 drone at (7, 8), reward: 19.90
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  1  0  1  0 -1 -1 -1 -1]
 [-1  0  1  0  1  1 -1 -1 -1 -1]
 [-1  1  1  1  1  1 -1 -1 -1 -1]
 [-1  1  1  1  1  1 -1 -1 -1 -1]
 [-1  1  1  0  1  0  0  0  1  1]
 [-1 -1 -1 -1 -1 -1  1  1  0  1]
 [-1 -1 -1 -1 -1 -1  1  1  1  1]
 [-1 -1 -1 -1 -1 -1  0  1  1  1]
 [-1 -1 -1 -1 -1 -1  0  1  1  1]]
Step 2: Placed 5x5 drone at (8, 3), reward: 19.90
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  1  0  1  0 -1 -1 -1 -1]
 [-1  0  1  0  1  1 -1 -1 -1 -1]
 [-1  1  1  1  1  1 -1 -1 -1 -1]
 [-1  1  1  1  1  1 -1 -1 -1 -1]
 [-1  1  1  0  1  0  0 

# Env 2: Env 1 + Static grid, high-priority zones, battery cost, discovering priority SAFE cells. Result≈14-53steps

In [4]:
PRIORITY = 2        # "high-value grids"
MOVE_COST = 1        # Each next action consumes 1 grid of power
COMM_RANGE = 3        # Single hop communication radius (Manhattan)

class AdvancedDroneEnv(DronePlacementEnv):

    def __init__(
        self,
        grid_size: int = 10,
        max_steps: int = 50,
        init_battery: int = 60,        # total power
        priority_prob: float = 0.1       # Random high-value lattice probability
    ):
        self.init_battery = init_battery
        self.priority_prob = priority_prob
        super().__init__(grid_size=grid_size, max_steps=max_steps)
        self.action_space = spaces.Discrete(grid_size * grid_size * 4)

    def reset(self, *, seed=None, options=None):
        obs, info = super().reset(seed=seed)
        self.battery = self.init_battery
        self.priority_map = self._init_priority_map()
        self.discovered_priority = np.zeros_like(self.priority_map, dtype=bool)
        return obs, info

    def _init_priority_map(self):
        # 1 = high‑priority SAFE
        pr_map = np.zeros_like(self.actual_env, dtype=np.int8)
        mask = (self.actual_env == SAFE) & (np.random.rand(*self.actual_env.shape) < self.priority_prob)
        pr_map[mask] = 1
        return pr_map

    def step(self, action):
        # Parse the action
        self.battery -= MOVE_COST
        role_bit = action % 2          # 0=explorer 1=relay
        pure_action = action // 2
        grid_action_space = self.grid_size * self.grid_size * 2
        assert pure_action < grid_action_space, "Action decode OOB"

        obs, base_reward, terminated, truncated, info = super().step(pure_action)

        # Discover the priority case
        new_priority = self._discover_cells()
        bonus_priority = 2 * new_priority
        base_reward += bonus_priority

        # Communication reward/penalty
        comm_ok = self._is_connected(role_bit, pure_action)
        base_reward += 1.0 if comm_ok else -1.0
        # Charge terminates
        if self.battery <= 0:
            truncated = True

        # End rewards to remaining power
        if terminated:
            base_reward += 0.05 * self.battery
        return obs, base_reward, terminated, truncated, info

    def _discover_cells(self):
        newly_found = 0
        undiscovered = ~self.discovered_priority & (self.global_grid == SAFE) & (self.priority_map == 1)
        newly_found = np.sum(undiscovered)
        self.discovered_priority |= undiscovered
        return newly_found

    # Communication connectivity
    def _is_connected(self, role_bit, pure_action):
        if role_bit == 1:
            pass

        # Collect positions of explorer drones
        explorers = []
        action_per_row = self.grid_size * 2
        x = pure_action // action_per_row
        y = (pure_action % action_per_row) // 2
        drone_type = pure_action % 2
        explorers.append((x, y))

        base = (0, 0)
        for ex in explorers:
            if abs(ex[0] - base[0]) + abs(ex[1] - base[1]) > COMM_RANGE:
                # If any explorer is too far from base (without a relay), assume disconnected
                return False
        return True

    def render(self):
        print(self.global_grid)

Random rollout reward: 65.25


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/env_checker.py:272: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [5]:
from stable_baselines3.common.env_util import make_vec_env

# Stage 1: Train on Small Grid
def train_stage1_small():
    print("Stage 1: Training on small grid")
    env = make_vec_env(lambda: AdvancedDroneEnv(grid_size=6, max_steps=30, init_battery=40), n_envs=4)

    model = PPO("MlpPolicy", env,
                verbose=1,
                device="cuda" if torch.cuda.is_available() else "cpu",
                policy_kwargs=dict(net_arch=[dict(pi=[64, 64], vf=[64, 64])]))
    model.learn(total_timesteps=100_000)
    model.save("ppo_smallmap_advanced_model_2")

    # Extract hidden layers
    encoder_state = {
        k: v for k, v in model.policy.mlp_extractor.state_dict().items()
        if not k.startswith("policy_net.0") and not k.startswith("value_net.0")
    }
    torch.save(encoder_state, "encoder_stage1.pt")
    print("Stage 1 model and encoder saved.")

# Stage 2: Train on Large Grid with Transferred Features
def train_stage2_big():
    print("\n Stage 2: Transfer learning on large grid")
    env = make_vec_env(lambda: AdvancedDroneEnv(grid_size=10, max_steps=60, init_battery=80), n_envs=4)

    model = PPO("MlpPolicy", env,
                verbose=1,
                device="cuda" if torch.cuda.is_available() else "cpu",
                policy_kwargs=dict(net_arch=[dict(pi=[64, 64], vf=[64, 64])]))

    # Load transferred hidden layers
    encoder_state = torch.load("encoder_stage1.pt")
    model.policy.mlp_extractor.load_state_dict(encoder_state, strict=False)

    model.learn(total_timesteps=300_000)
    model.save("ppo_bigmap_advanced_model_2")
    print("Stage 2 training complete.")

if __name__ == "__main__":
    train_stage1_small()
    train_stage2_big()

Stage 1: Training on small grid
Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.9     |
|    ep_rew_mean     | 45.3     |
| time/              |          |
|    fps             | 2317     |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 10.9        |
|    ep_rew_mean          | 49          |
| time/                   |             |
|    fps                  | 1222        |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.026783219 |
|    clip_fraction        | 0.21        |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.95       |
|    explained_varianc

In [93]:
def run_trained_model_advanced():
    model = PPO.load("ppo_bigmap_advanced_model_2")
    # same environment as in training
    env = AdvancedDroneEnv(grid_size=10, max_steps=60, init_battery=80)
    obs, _ = env.reset()

    for step in range(env.max_steps):
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, _ = env.step(action)
        x = action // 20
        y = (action % 20) // 2
        drone_type = "3x3" if action % 2 == 0 else "5x5"
        print(f"Step {step}: Placed {drone_type} drone at ({x}, {y}), reward: {reward:.2f}")
        env.render()
        if terminated or truncated:
            print("Mission Complete: All cells explored!")
            break
        if truncated:
            print("Max steps reached.")
            break

if __name__ == "__main__":
    run_trained_model_advanced()

Step 0: Placed 3x3 drone at (15, 7), reward: 24.90
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1  0  1  1  0]
 [-1 -1 -1 -1 -1 -1  1  0  1  1]
 [-1 -1 -1 -1 -1 -1  1  1  1  1]
 [-1 -1 -1 -1 -1 -1  1  1  1  1]
 [-1 -1 -1 -1 -1 -1  1  1  1  1]]
Step 1: Placed 3x3 drone at (12, 9), reward: 23.60
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1  1  1  1  1  0 -1 -1 -1]
 [-1 -1  1  1  1  1  0  1  1  0]
 [-1 -1  1  1  1  1  1  0  1  1]
 [-1 -1  1  0  0  0  1  1  1  1]
 [-1 -1  1  1  1  1  1  1  1  1]
 [-1 -1 -1 -1 -1 -1  1  1  1  1]]
Step 2: Placed 5x5 drone at (2, 1), reward: 14.90
[[ 1  1  0 -1 -1 -1 -1 -1 -1 -1]
 [ 0  1  1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1  1  1  1  1  0 -1 -1 -1]
 [-1 -1  1  1  1  1  

# Env 3: Env 2 + Must land in safe zone to get final reward, Dynamic grid, Parallel agents. Result≈9-59steps

In [7]:
# dynamic_marl_env.py
import numpy as np
import random
from gymnasium import spaces

# 1. Single-agent extension: dynamic obstacles + landing zones
class DynamicLandingEnv(AdvancedDroneEnv):
    def __init__(
        self,
        grid_size=10,
        max_steps=60,
        init_battery=80,
        dynamic_flip_prob=0.02,      # Probability for cell to flip state
        landing_buffer=1
    ):
        self.dynamic_flip_prob = dynamic_flip_prob
        self.landing_buffer = landing_buffer
        super().__init__(grid_size=grid_size, max_steps=max_steps, init_battery=init_battery)

    def reset(self, *, seed=None, options=None):
        obs, info = super().reset(seed=seed)
        self.landable_set = self._compute_landable()
        return obs, info

    def step(self, action):
        # First update dynamic obstacles
        self._update_dynamic()

        obs, r, terminated, truncated, info = super().step(action)

        # mission complete or timeout
        if terminated or truncated:
            if hasattr(self, "last_xy"):
                landed_ok = self.last_xy in self.landable_set
                # +5 / -5 landing bonus
                r += 5.0 if landed_ok else -5.0
                info["landed_ok"] = landed_ok
        return obs, r, terminated, truncated, info

    # record the final coordinates
    def step(self, action):
        role_bit = action % 2
        pure_action = action // 2
        a_per_row = self.grid_size * 2
        x = pure_action // a_per_row
        y = (pure_action % a_per_row) // 2
        self.last_xy = (x, y)

        # Continue with parent logic (after dynamic obstacle update)
        return super().step(action)

    # Dynamic obstacle update
    def _update_dynamic(self):
        flip_mask = np.random.rand(*self.actual_env.shape) < self.dynamic_flip_prob
        # Flip between SAFE and OBSTACLE
        self.actual_env[flip_mask & (self.actual_env == 1)] = 0
        self.actual_env[flip_mask & (self.actual_env == 0)] = 1

    # Compute landing zones
    def _compute_landable(self):
        landable = set()
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                if self.actual_env[x, y] != 1:      # Must be SAFE
                    continue
                ok = True
                # All surrounding cells within the buffer must be SAFE
                for dx in range(-self.landing_buffer, self.landing_buffer + 1):
                    for dy in range(-self.landing_buffer, self.landing_buffer + 1):
                        gx, gy = x + dx, y + dy
                        if 0 <= gx < self.grid_size and 0 <= gy < self.grid_size:
                            if self.actual_env[gx, gy] == 0:
                                ok = False; break
                    if not ok: break
                if ok:
                    landable.add((x, y))
        # Guarantee at least (0,0) is a valid landing zone
        landable.add((0, 0))
        return landable


# Multi-agent PettingZoo environment (Explorer vs Relay)
from pettingzoo.utils.env import ParallelEnv
from pettingzoo.utils import wrappers

EXPLORER = 0
RELAY    = 1

class MultiDroneEnv(ParallelEnv):
    """
    • n_explorers perform scanning
    • n_relays act as static relays (do not scan or move)
      → You can extend this to allow relays to move
    Action structure is identical to single-agent, but separated by agent
    Observation = shared global grid
    """
    metadata = {"name": "multi_drone_v0"}
    render_mode = None

    def __init__(
        self,
        n_explorers=2,
        n_relays=1,
        grid_size=10,
        max_steps=60,
    ):
        super().__init__()
        self.grid_size = grid_size
        self.max_steps = max_steps
        self.n_explorers = n_explorers
        self.n_relays = n_relays

        # Underlying single-agent environment
        self.base_env = DynamicLandingEnv(grid_size=grid_size,max_steps=max_steps)

        # agent ids
        self.agent_names = (
            [f"explorer_{i}" for i in range(n_explorers)] +
            [f"relay_{i}"     for i in range(n_relays)]
        )
        self.possible_agents = list(self.agent_names)

        self.action_spaces = {agent: self.base_env.action_space
                              for agent in self.agent_names}
        self.observation_spaces = {agent: self.base_env.observation_space
                                   for agent in self.agent_names}

    # PettingZoo API
    def reset(self, seed=None, options=None):
        obs, _ = self.base_env.reset(seed=seed)
        self.steps = 0
        self.agents = list(self.agent_names)
        info = {a: {} for a in self.agent_names}
        return {a: obs.copy() for a in self.agent_names}, info



    def step(self, actions):
        """
        actions: {agent_name: action_int}
        We only execute one explorer’s action for simplicity;
        You can change this to queue all actions or use turn-based logic.
        """
        self.steps += 1
        exec_agent = f"explorer_{self.steps % self.n_explorers}"
        action_to_exec = actions.get(exec_agent,
                                     self.base_env.action_space.sample())
        obs, r, term, trunc, info = self.base_env.step(int(action_to_exec))

        # Same reward is shared by all agents
        rewards = {a: r for a in self.agent_names}
        terminations = {a: term for a in self.agent_names}
        truncations  = {a: trunc for a in self.agent_names}
        infos        = {a: info for a in self.agent_names}
        observations = {a: obs.copy() for a in self.agent_names}

        # Global termination
        if term or trunc or self.steps >= self.max_steps:
            terminations = {a: True for a in self.agent_names}
            truncations  = {a: trunc for a in self.agent_names}
            self.agents = []

        return observations, rewards, terminations, truncations, infos

    def render(self):
        self.base_env.render()

    def close(self):
        pass

# Optionally wrap with supersuit utilities
def ss_env(**kwargs):
    raw_env = MultiDroneEnv(**kwargs)
    wrapped = ss.pad_observations_v0(ss.pad_action_space_v0(raw_env))
    return wrapped

single‑agent random reward: 46.05


In [8]:
from stable_baselines3 import PPO
import supersuit as ss
import torch

# Phase 1: Small Grid
def train_stage1_small_marl():
    print(" Phase 1: Small grid MARL training")

    def ss_env_small():
        raw_env = MultiDroneEnv(grid_size=6, max_steps=30)
        return ss.pad_observations_v0(ss.pad_action_space_v0(raw_env))

    petting_env = ss_env_small()
    vec_env = ss.pettingzoo_env_to_vec_env_v1(petting_env)
    vec_env = ss.concat_vec_envs_v1(vec_env, num_vec_envs=4, num_cpus=2, base_class="stable_baselines3")

    model = PPO("MlpPolicy", vec_env,
                device="cuda" if torch.cuda.is_available() else "cpu",
                verbose=1,
                policy_kwargs=dict(net_arch=[dict(pi=[64, 64], vf=[64, 64])]))

    model.learn(total_timesteps=500_000)
    model.save("ppo_marl_stage1_small")

    encoder_state = {
        k: v for k, v in model.policy.mlp_extractor.state_dict().items()
        if not k.startswith("policy_net.0") and not k.startswith("value_net.0")
    }
    torch.save(encoder_state, "encoder_marl_stage1.pt")
    print(" Stage 1 model & encoder saved.")


# Phase 2: Large Grid
def train_stage2_big_marl():
    print("\n Phase 2: Large grid MARL fine-tuning")

    def ss_env_big():
        raw_env = MultiDroneEnv(grid_size=10, max_steps=60)
        return ss.pad_observations_v0(ss.pad_action_space_v0(raw_env))

    petting_env = ss_env_big()
    vec_env = ss.pettingzoo_env_to_vec_env_v1(petting_env)
    vec_env = ss.concat_vec_envs_v1(vec_env, num_vec_envs=8, num_cpus=4, base_class="stable_baselines3")

    model = PPO("MlpPolicy", vec_env,
                device="cuda" if torch.cuda.is_available() else "cpu",
                verbose=1,
                policy_kwargs=dict(net_arch=[dict(pi=[64, 64], vf=[64, 64])]))

    encoder_state = torch.load("encoder_marl_stage1.pt")
    model.policy.mlp_extractor.load_state_dict(encoder_state, strict=False)

    model.learn(total_timesteps=1000_000)
    model.save("ppo_marl_drones_gpu_3")
    print("Stage 2 MARL training complete.")

if __name__ == "__main__":
    train_stage1_small_marl()
    train_stage2_big_marl()


📘 Phase 1: Small grid MARL training


/usr/local/lib/python3.11/dist-packages/pettingzoo/utils/env.py:370: UserWarning: Your environment should override the action_space function. Attempting to use the action_spaces dict attribute.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pettingzoo/utils/env.py:358: UserWarning: Your environment should override the observation_space function. Attempting to use the observation_spaces dict attribute.
  warnings.warn(


Using cuda device
------------------------------
| time/              |       |
|    fps             | 2684  |
|    iterations      | 1     |
|    time_elapsed    | 9     |
|    total_timesteps | 24576 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1379        |
|    iterations           | 2           |
|    time_elapsed         | 35          |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.021091426 |
|    clip_fraction        | 0.186       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.96       |
|    explained_variance   | 0.0181      |
|    learning_rate        | 0.0003      |
|    loss                 | 9.78        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0301     |
|    value_loss           | 65.8        |
-----------------------------------------
---------

In [97]:
from stable_baselines3 import PPO
import supersuit as ss

def run_marl_rollout():
    model = PPO.load("ppo_marl_drones_gpu_3")
    env = ss_env(grid_size=10)  # already defined globally
    obs, _ = env.reset()
    step = 0

    while env.agents:
        print(f"\n Step {step}")
        actions = {}

        for agent, observation in obs.items():
            action, _ = model.predict(observation, deterministic=True)
            actions[agent] = action

        obs, rewards, terminations, truncations, infos = env.step(actions)

        for agent in rewards:
            x = actions[agent] // 20
            y = (actions[agent] % 20) // 2
            drone_type = "3x3" if actions[agent] % 2 == 0 else "5x5"
            print(f"{agent}: {drone_type} drone at ({x}, {y}) → reward: {rewards[agent]:.2f}")

        env.render()
        step += 1

        if all(terminations.values()) or all(truncations.values()):
            print("\n Episode finished!")
            break

run_marl_rollout()


 Step 0
explorer_0: 5x5 drone at (6, 0) → reward: 7.10
explorer_1: 5x5 drone at (6, 0) → reward: 7.10
relay_0: 5x5 drone at (6, 0) → reward: 7.10
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0  1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  0 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]

 Step 1
explorer_0: 3x3 drone at (13, 1) → reward: 31.60
explorer_1: 3x3 drone at (13, 1) → reward: 31.60
relay_0: 3x3 drone at (13, 1) → reward: 31.60
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 0  1 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  0 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1 -1  1  0  0  1  0 -1 -1]
 [-1 -1 -1  1  1  0  1  0 -1 -1]
 [-1 -1 -1  1  0  1  1  1 -1 -1]
 [-1 -1 -1  1  1  1  1  1 -1 -1]
 [-1 -1 -1  1  1  1  1  1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]

 Step 2
explorer_0: 3x3 drone at (5, 